# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

# Lesson: Pointer Chain Operations

In [1]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 2.1MB 6.3MB/s 
     |████████████████████████████████| 276kB 43.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 28691 (delta 1), reused 3 (delta 0), pack-reused 28684
Receiving objects: 100% (28691/28691), 32.07 MiB | 17.04 MiB/s, done.
Resolving deltas: 100% (19020/19020), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |████████████████████████████████| 389kB 6.4MB/s 
ERROR: syft 0.1.21a1 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5.6 whi

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy

W0723 18:21:40.604112 140665277544320 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0723 18:21:40.623174 140665277544320 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker=sy.VirtualWorker(hook, id="secure_worker")
bob.add_workers([alice,secure_worker])
alice.add_workers([bob,secure_worker])
secure_worker.add_workers([alice,bob])

W0723 18:21:54.662210 140665277544320 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0723 18:21:54.664159 140665277544320 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0723 18:21:54.665334 140665277544320 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0723 18:21:54.666596 140665277544320 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0723 18:21:54.668752 140665277544320 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0723 18:21:54.670842 140665277544320 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = True

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}
torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")
federated_train_loader = sy.FederatedDataLoader(
                          datasets.MNIST('../data', train=True, download=True,
                          transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,), (0.3081,))])).federate((bob, alice)),
                          batch_size=args.batch_size, shuffle=True,**kwargs)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

0it [00:00, ?it/s]

9920512it [00:01, 8747873.07it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 135542.44it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

...

1654784it [00:00, 2208700.60it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 51446.36it/s]            


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


W0723 18:23:22.674018 140665277544320 dataloader.py:197] The following options are not supported: num_workers: 1, pin_memory: True


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(4*4*64, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        # print(x)
        x = F.relu(self.conv1(x))
        # print(x)
        x = F.max_pool2d(x, 2, 2)
        # print(x)
        x = F.relu(self.conv2(x))
        # print(x)
        x = F.max_pool2d(x, 2, 2)
        # print(x)
        x = x.view(-1, 4*4*64)
        # print(x)
        x = F.relu(self.fc1(x))
        # print(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
test_loader = torch.utils.data.DataLoader(
                       datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))])),
                       batch_size=args.test_batch_size, shuffle=True)
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [0]:
model = Net().cuda()
model.train()
model_bob = model.copy().send(bob)
model_alice= model.copy().send(alice)
optimizer_bob = optim.SGD(params=model_bob.parameters(), lr=args.lr)
optimizer_alice = optim.SGD(params=model_alice.parameters(), lr=args.lr)


In [9]:
for epoch in range(1, args.epochs + 1):

    for idx,(data, target) in enumerate(federated_train_loader):

        if data.location == bob:
            model_bob.train()
            data, target = data.to(device), target.to(device)
            optimizer_bob.zero_grad()
            output_bob = model_bob(data)
            loss_bob = F.nll_loss(output_bob, target)
            loss_bob.backward()
            optimizer_bob.step()
            loss_bob = loss_bob.get().data

        elif data.location == alice:
            model_alice.train()
            data, target = data.to(device), target.to(device)
            optimizer_alice.zero_grad()
            output_alice = model_alice(data)
            loss_alice = F.nll_loss(output_alice, target)
            loss_alice.backward()
            optimizer_alice.step()
            loss_alice = loss_alice.get().data
    print('Train Epoch: {} \tAlice Loss: {:.6f} \tbob Loss: {:.6f}'.format(
        epoch, loss_alice.item(), loss_bob.item()))
    with torch.no_grad():
        pram1=[pram for pram in model_bob.parameters()]
        pram2=[pram for pram in model_alice.parameters()]
        model_alice.move(secure_worker)
        model_bob.move(secure_worker)

        for i in range(len(pram1)):
            pram1[i].set_(((pram1[i] + pram2[i]) * 0.5).float().to(device))
            pram2[i].set_(((pram1[i] + pram2[i]) * 0.5).float().to(device))

        model_bob= model_bob.get()
        test(args, model_bob, device, test_loader)
        model_alice= model_alice.get()
        test(args, model_alice, device, test_loader)
        model_bob = model_bob.send(bob)
        model_alice = model_alice.send(alice)

            

Train Epoch: 1 	Alice Loss: 0.206999 	bob Loss: 0.194432

Test set: Average loss: 0.2663, Accuracy: 9208/10000 (92%)


Test set: Average loss: 0.2826, Accuracy: 9137/10000 (91%)

Train Epoch: 2 	Alice Loss: 0.150931 	bob Loss: 0.241952

Test set: Average loss: 0.1667, Accuracy: 9520/10000 (95%)


Test set: Average loss: 0.1717, Accuracy: 9460/10000 (95%)

Train Epoch: 3 	Alice Loss: 0.030590 	bob Loss: 0.190386

Test set: Average loss: 0.1114, Accuracy: 9663/10000 (97%)


Test set: Average loss: 0.1181, Accuracy: 9646/10000 (96%)

Train Epoch: 4 	Alice Loss: 0.051556 	bob Loss: 0.106082

Test set: Average loss: 0.0991, Accuracy: 9697/10000 (97%)


Test set: Average loss: 0.1004, Accuracy: 9707/10000 (97%)

Train Epoch: 5 	Alice Loss: 0.067597 	bob Loss: 0.151261

Test set: Average loss: 0.0871, Accuracy: 9737/10000 (97%)


Test set: Average loss: 0.0822, Accuracy: 9753/10000 (98%)

Train Epoch: 6 	Alice Loss: 0.134932 	bob Loss: 0.039990

Test set: Average loss: 0.0685, Accuracy: 9778/

In [0]:
! nvidia-smi

Mon Jul 22 10:40:48 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch 

In [0]:
torch.__version__

'1.1.0'